<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://biblioteca.uah.es/imgs/logo11.png", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">EN26 - HERRAMIENTAS DE ANÁLISIS · PEC2</p>
    <p style="margin: 0; text-align:right;">ENTORNOS DE ANÁLISIS DE DATOS (PYTHON)  </p>
<p style="margin: 0; text-align:right;">2018-2019 · Máster universitario en Ciencia de datos (Data science)</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# Objetivo de la prueba
<br>
<div>
    <p>El ejercicio consiste en entregar un Notebook con los siguientes puntos:</p>
<ol>
<li>Utilizar el API REST de StackEchange (<a href="https://api.stackexchange.com/">api.stackexchange.com</a>) para obtener una muestra de más de 1000 preguntas sobre un tag concreto. Por ejemplo, el tag podría ser "python". Convertir esos datos en un DataFrame con al menos los siguientes elementos: identificador de usuario, identificador de pregunta, título, si se ha contestado o no, número de veces que se ha visto y fecha de creación. Sobre ese DataFrame , hacer lo siguiente:</li>
<ol>
<li>Buscar la pregunta más antigua no contestada.</li>
<li>Obtener y dibujar la distribución de las visitas.</li>
</ol>
<li>Sobre el DataFrame anterior, incluir columnas adicionales obteniendo información del API sobre la respuesta más valorada a cada pregunta, con al menos el identificador de la respuesta, la puntuación, la fecha en que se creó y el identificador del usuario que respondió.</li>
<li>Sobre el DataFrame anterior, extraer todos los usuarios diferentes que aparecen, y obtener el usuario de mayor y de menor reputación.</li>
</ol>
</div>

# Obtención del conjunto de datos del repositorio

<p>En primer lugar procedemos a realizar la conexión con <a href="https://api.stackexchange.com/">api.stackexchange.com</a>.
    Para ello vamos a utilizar la librería <b>'stackapi'</b>, cuyos datos de utilización y manual operativo se pueden ver en <a href="https://stackapi.readthedocs.io">stackapi.readthedocs.io</a>.</p>

## Instalación de la librería
<p>Como no sabemos si está o no instalada en el sistema de librerias de Python esta librería, en primer lugar procedemos a su instalación.</p>


In [1]:
!pip install stackapi

## Carga de la librería y del site

<p>A continuación realizamos la carga de la librería y del site sobre el que vamos a trabajar para resonder las cuestiones planteadas. En este caso el site es <b>'stackoverflow'</b>.</p>

In [2]:
from stackapi import StackAPI

SITE = StackAPI('stackoverflow')

## Definición de los parámetros de búsqueda y carga de datos

<p>Seguidamente realizamos la definición de los parámetros de búsqueda que necesitamos y procedemos a la carga de datos contra el site.</p>
<p>Atendiendo a los requerimientos de la prueba, procederemos a definir los parámetros de configuración de límite maximo de carga de datos:</p>

<ul>
    <li><b>page_size:</b> tamaño máximo de pagina. Por defecto el tamaño de carga es de 100</li>
    <li><b>max_pages:</b> número de paginas maximo a cargar</li>
</ul>

<p>El multiplicador de ambos parámetros nos dará el tamaño máximo de carga. En nuestro caso sería de 2.000 objetos.</p>


In [3]:
SITE.page_size = 100
SITE.max_pages = 20

questions = SITE.fetch('questions', min=20, tagged='python', sort='votes')
questions

{'backoff': 0,
 'has_more': True,
 'page': 20,
 'quota_max': 300,
 'quota_remaining': 300,
 'total': 0,
 'items': [{'tags': ['python', 'iterator', 'generator', 'yield', 'coroutine'],
   'owner': {'reputation': 51619,
    'user_id': 18300,
    'user_type': 'registered',
    'accept_rate': 92,
    'profile_image': 'https://i.stack.imgur.com/jcyI4.jpg?s=128&g=1',
    'display_name': 'Alex. S.',
    'link': 'https://stackoverflow.com/users/18300/alex-s'},
   'is_answered': True,
   'view_count': 1949000,
   'protected_date': 1360547307,
   'accepted_answer_id': 231855,
   'answer_count': 40,
   'score': 8943,
   'last_activity_date': 1549425475,
   'creation_date': 1224800471,
   'last_edit_date': 1513279312,
   'question_id': 231767,
   'link': 'https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do',
   'title': 'What does the &quot;yield&quot; keyword do?'},
  {'tags': ['python',
    'operators',
    'ternary-operator',
    'conditional-operator',
    'python-2.5'],
 

## Análisis de los datos obtenidos
### Características generales

<p>En primer lugar vamos a verificar qué claves están disponibles en el objeto de tipo diccionario que hemos obtenido con nuestra consulta.</p>

In [4]:
print(questions.keys())
type(questions['items'])

dict_keys(['backoff', 'has_more', 'page', 'quota_max', 'quota_remaining', 'total', 'items'])


list

### Extracción de items y verificación de información

<p>Dado que hemos verificado que nuestro diccionario contenia 7 claves, y una de ellas (<b>items</b>) es la que previsiblemente contiene los datos que vamos a necesitar, procedemos a extraer los datos de <b>items</b> en un formato <i>lista</i>.</p>
<p>Esta lista contiene a su vez el conjunto completo de 2000 diccionarios. Vamos a analizar uno para ver como podemos extraer la información que necesitamos.</p>

In [5]:
qlist = questions['items']  
qlist[0]

{'tags': ['python', 'iterator', 'generator', 'yield', 'coroutine'],
 'owner': {'reputation': 51619,
  'user_id': 18300,
  'user_type': 'registered',
  'accept_rate': 92,
  'profile_image': 'https://i.stack.imgur.com/jcyI4.jpg?s=128&g=1',
  'display_name': 'Alex. S.',
  'link': 'https://stackoverflow.com/users/18300/alex-s'},
 'is_answered': True,
 'view_count': 1949000,
 'protected_date': 1360547307,
 'accepted_answer_id': 231855,
 'answer_count': 40,
 'score': 8943,
 'last_activity_date': 1549425475,
 'creation_date': 1224800471,
 'last_edit_date': 1513279312,
 'question_id': 231767,
 'link': 'https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do',
 'title': 'What does the &quot;yield&quot; keyword do?'}

<p>Como se puede verificar, cada uno de los elementos de la variable <b>qlist</b> es a su vez un diccionario. A continuación, vamos a ver algunos de los elementos de este diccionario que nos pueden ser útilies ala hora de conformar nuestro dataframe.<p>

In [6]:
print('IDuser = ', qlist[0]['owner']['user_id'])
print('IDquestion = ', qlist[0]['question_id'])
print('Titulo de la pregunta = ', qlist[0]['title'])
print('Respondida = ', qlist[0]['is_answered'])
print('Contador vistas = ', qlist[0]['view_count'])
print('Fecha Creación = ', qlist[0]['creation_date'])
print('Reputacion = ', qlist[0]['owner']['reputation'])
print('Tipo de usuario = ', qlist[0]['owner']['user_type'])
print('Contador de respuestas = ', qlist[0]['answer_count'])

IDuser =  18300
IDquestion =  231767
Titulo de la pregunta =  What does the &quot;yield&quot; keyword do?
Respondida =  True
Contador vistas =  1949000
Fecha Creación =  1224800471
Reputacion =  51619
Tipo de usuario =  registered
Contador de respuestas =  40


### Extracción de datos y creación del dataframe

<p>Con la información extraida anteriormente, podemos ver qu elo mas práctico es crear un bucle que recorra el listado y extraiga los datos que vayamos aincorporar al dataframe.</p>
<p>Es importante saber que dentro de cada elemento de tipo <i>dict</i> del listado puede haber o no las claves que necesitemos. Por ello utilizaremos la función <b>get()</b> que devuelve <i>none</i> en el caso de que no este la clave entre los datos recuperados. Este valor <i>none</i> se incorporará al dataframe como indicador de dato faltante: <i>NA</i><p>
<p>Por último, aunque solo se requieren por definición de la práctica 6 variables, para nuestro propósito vamos a recoger datos de 9 variables:</p>
<ul>
<li><b>Identificador de usuario (IDUser):</b> Se extrae de ['owner']['user_id']</li>
<li><b>Identificador de pregunta (IDQuestion):</b> Se extrae de ['question_id']</li>
<li><b>Titulo de la pregunta (Title):</b> Se extrae de ['title']</li>
<li><b>Identificador de pregunta respondida (IsAnswered):</b> Se extrae de ['is_answered']</li>
<li><b>Contador vistas (ViewCount):</b> Se extrae de ['view_count']</li>
<li><b>Fecha Creación (CreationDate):</b> Se extrae de ['creation_date']</li>
<li><b>Valor de Reputacion (Reputation):</b> Se extrae de ['reputation']</li>
<li><b>Tipo de usuario (UserType):</b> Se extrae de ['owner']['user_type']</li>
<li><b>Contador de respuestas (AnswerCount):</b> Se extrae de ['answer_count']  </li>  
</ul>

In [8]:
import pandas as pd
df = pd.DataFrame(qlist)
df.head()

,accepted_answer_id,answer_count,closed_date,closed_reason,community_owned_date,creation_date,is_answered,last_activity_date,last_edit_date,link,locked_date,migrated_from,owner,protected_date,question_id,score,tags,title,view_count
0,231855.0,40,NaN,NaN,NaN,1224800471,True,1549425475,1.513279e+09,https://stackoverflow.com/questions/231767/wha...,NaN,NaN,"{'reputation': 51619, 'user_id': 18300, 'user_...",1.360547e+09,231767,8943,"[python, iterator, generator, yield, coroutine]",What does the &quot;yield&quot; keyword do?,1949000
1,394814.0,26,NaN,NaN,1.358001e+09,1230366738,True,1550125907,1.550126e+09,https://stackoverflow.com/questions/394809/doe...,NaN,NaN,"{'reputation': 33575, 'user_id': 51518, 'user_...",1.370841e+09,394809,4965,"[python, operators, ternary-operator, conditio...",Does Python have a ternary conditional operator?,1598641
2,100146.0,15,NaN,NaN,NaN,1221804646,True,1546883483,1.515713e+09,https://stackoverflow.com/questions/100003/wha...,NaN,NaN,"{'reputation': 357390, 'user_id': 9951, 'user_...",1.350586e+09,100003,4953,"[python, oop, metaclass, python-datamodel]",What are metaclasses in Python?,670722
3,NaN,40,NaN,NaN,NaN,1221656100,True,1549618593,1.541440e+09,https://stackoverflow.com/questions/82831/how-...,NaN,NaN,"{'reputation': 26375, 'user_id': 15616, 'user_...",1.370841e+09,82831,4813,"[python, file, file-exists]",How do I check whether a file exists without e...,3366917
4,419185.0,27,NaN,NaN,NaN,1231301460,True,1549814688,1.527241e+09,https://stackoverflow.com/questions/419163/wha...,NaN,NaN,"{'reputation': 33575, 'user_id': 51518, 'user_...",1.365246e+09,419163,4704,"[python, namespaces, main, python-module, idioms]",What does if __name__ == &quot;__main__&quot;:...,2100198


<p>Como podemos ver en la tabla previa, hay algunas variables que podemos tomar directamente para nuestro dataframe y otras que no es posible, ya que la variable <b>owner</b> es un diccionario en sí misma.</p>
<p>En primer lugar vamos a crear un segundo dataframe a partir de la variable <b>owner</b>:</p>

In [12]:
owner = pd.DataFrame(df['owner'].values.tolist())
owner.head()

,accept_rate,display_name,link,profile_image,reputation,user_id,user_type
0,92.0,Alex. S.,https://stackoverflow.com/users/18300/alex-s,https://i.stack.imgur.com/jcyI4.jpg?s=128&g=1,51619.0,18300.0,registered
1,86.0,Devoted,https://stackoverflow.com/users/51518/devoted,https://www.gravatar.com/avatar/eb5a95cb1be342...,33575.0,51518.0,registered
2,78.0,e-satis,https://stackoverflow.com/users/9951/e-satis,https://www.gravatar.com/avatar/1449ec95d8c114...,357390.0,9951.0,registered
3,83.0,spence91,https://stackoverflow.com/users/15616/spence91,https://www.gravatar.com/avatar/1d502f7932af0a...,26375.0,15616.0,registered
4,86.0,Devoted,https://stackoverflow.com/users/51518/devoted,https://www.gravatar.com/avatar/eb5a95cb1be342...,33575.0,51518.0,registered


In [13]:
owner.columns.values

array(['accept_rate', 'display_name', 'link', 'profile_image',
       'reputation', 'user_id', 'user_type'], dtype=object)

In [16]:
owner = owner.drop(['accept_rate', 'display_name', 'link', 'profile_image'], axis=1)
owner.head()

,reputation,user_id,user_type
0,51619.0,18300.0,registered
1,33575.0,51518.0,registered
2,357390.0,9951.0,registered
3,26375.0,15616.0,registered
4,33575.0,51518.0,registered


<p>A continuación vamos a realizar el mismo proceso pero con el dataframe principal <b>df</b></p>

In [17]:
df.columns.values

array(['accepted_answer_id', 'answer_count', 'closed_date',
       'closed_reason', 'community_owned_date', 'creation_date',
       'is_answered', 'last_activity_date', 'last_edit_date', 'link',
       'locked_date', 'migrated_from', 'owner', 'protected_date',
       'question_id', 'score', 'tags', 'title', 'view_count'],
      dtype=object)

In [18]:
df = df.drop(['accepted_answer_id', 'closed_date', 'closed_reason', 'community_owned_date', 
       'last_activity_date', 'last_edit_date', 'link', 'locked_date', 'migrated_from', 
         'owner', 'protected_date', 'score', 'tags'], axis=1)
df.head()

,answer_count,creation_date,is_answered,question_id,title,view_count
0,40,1224800471,True,231767,What does the &quot;yield&quot; keyword do?,1949000
1,26,1230366738,True,394809,Does Python have a ternary conditional operator?,1598641
2,15,1221804646,True,100003,What are metaclasses in Python?,670722
3,40,1221656100,True,82831,How do I check whether a file exists without e...,3366917
4,27,1231301460,True,419163,What does if __name__ == &quot;__main__&quot;:...,2100198


<p>Finalmente, vamos a proceder a concatener ambos dataframes segun el indice del dataframe <b>df</b>, ya que la correspondencia es completa entre ambos dataframes en su indice respectivo.</p>

In [22]:
dfPython = pd.concat([df, owner], axis=1, join_axes=[df.index])
dfPython.head()

,answer_count,creation_date,is_answered,question_id,title,view_count,reputation,user_id,user_type
0,40,1224800471,True,231767,What does the &quot;yield&quot; keyword do?,1949000,51619.0,18300.0,registered
1,26,1230366738,True,394809,Does Python have a ternary conditional operator?,1598641,33575.0,51518.0,registered
2,15,1221804646,True,100003,What are metaclasses in Python?,670722,357390.0,9951.0,registered
3,40,1221656100,True,82831,How do I check whether a file exists without e...,3366917,26375.0,15616.0,registered
4,27,1231301460,True,419163,What does if __name__ == &quot;__main__&quot;:...,2100198,33575.0,51518.0,registered


In [23]:
dfPython.columns.values

array(['answer_count', 'creation_date', 'is_answered', 'question_id',
       'title', 'view_count', 'reputation', 'user_id', 'user_type'],
      dtype=object)

In [24]:
sequence = ['user_id', 'question_id', 'title', 'is_answered',
            'view_count', 'creation_date', 'reputation',
            'user_type', 'answer_count']

dfPython = dfPython.reindex(columns=sequence)
dfPython.head()

,user_id,question_id,title,is_answered,view_count,creation_date,reputation,user_type,answer_count
0,18300.0,231767,What does the &quot;yield&quot; keyword do?,True,1949000,1224800471,51619.0,registered,40
1,51518.0,394809,Does Python have a ternary conditional operator?,True,1598641,1230366738,33575.0,registered,26
2,9951.0,100003,What are metaclasses in Python?,True,670722,1221804646,357390.0,registered,15
3,15616.0,82831,How do I check whether a file exists without e...,True,3366917,1221656100,26375.0,registered,40
4,51518.0,419163,What does if __name__ == &quot;__main__&quot;:...,True,2100198,1231301460,33575.0,registered,27


In [25]:
sequence = {'user_id': 'IDUser', 
            'question_id': 'IDQuestion', 
            'title': 'Title', 
            'is_answered': 'IsAnswered',
            'view_count': 'ViewCount', 
            'creation_date': 'CreationDate', 
            'reputation': 'Reputation',
            'user_type': 'UserType', 
            'answer_count': 'AnswerCount'}

dfPython = dfPython.rename(columns = sequence) 
dfPython.head()

,IDUser,IDQuestion,Title,IsAnswered,ViewCount,CreationDate,Reputation,UserType,AnswerCount
0,18300.0,231767,What does the &quot;yield&quot; keyword do?,True,1949000,1224800471,51619.0,registered,40
1,51518.0,394809,Does Python have a ternary conditional operator?,True,1598641,1230366738,33575.0,registered,26
2,9951.0,100003,What are metaclasses in Python?,True,670722,1221804646,357390.0,registered,15
3,15616.0,82831,How do I check whether a file exists without e...,True,3366917,1221656100,26375.0,registered,40
4,51518.0,419163,What does if __name__ == &quot;__main__&quot;:...,True,2100198,1231301460,33575.0,registered,27


<p>Ahora ya tenemso el dataframe dfPython con als variables que hemos considerado al comienzo, en el orden que indicabamos y les hemos ajustado el nombre.</p>

## Buscar la pregunta más antigua no contestada
<p>Ahora vamos a tratar de encontrar en el dataframe la pregunta mas antigua no contestada.</p>